# 🟠 Lesson 8: Guidance Math (CFG)

Welcome to **Module 3: Controlling Generation**. We start with the most important dial: **Guidance Scale**.

### What is CFG?
**Classifier-Free Guidance** is a hack. The model generates TWO predictions at once:
1.  **Unconditioned**: "Draw whatever you want" (input is empty string `""`).
2.  **Conditioned**: "Draw a red apple" (input is your prompt).

The formula is:
$$ \text{Final} = \text{Uncond} + \text{Weight} \times (\text{Cond} - \text{Uncond}) $$

We calculate the vector difference ("The concept of Red Apple") and exaggerate it.

In [ ]:
# 1. Setup
import notebook_utils
project_root, device, dtype = notebook_utils.setup_notebook()

import numpy as np
import matplotlib.pyplot as plt

## 1. Visualizing CFG Vectors

Since we can't visualize 768 dimensions, let's visualize 2 dimensions.

In [ ]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def plot_cfg(weight):
    # 1. Uncond (Generic image)
    v_uncond = np.array([2, 1])
    
    # 2. Cond (Your prompt)
    v_cond = np.array([3, 4])
    
    # 3. Difference (The Concept)
    v_diff = v_cond - v_uncond
    
    # 4. Result
    v_final = v_uncond + weight * v_diff
    
    # Plot
    plt.figure(figsize=(6,6))
    plt.xlim(0, 15)
    plt.ylim(0, 15)
    plt.grid(True, alpha=0.3)
    
    # Draw Arrows
    opts = dict(head_width=0.5, head_length=0.5)
    plt.arrow(0, 0, *v_uncond, fc='gray', ec='gray', label='Unconditioned', **opts)
    plt.arrow(0, 0, *v_cond, fc='blue', ec='blue', label='Conditioned', **opts)
    plt.arrow(*v_uncond, *v_diff, fc='green', ec='green', label='Difference', **opts)
    plt.arrow(0, 0, *v_final, fc='red', ec='red', label=f'Final (CFG {weight})', **opts)
    
    plt.legend()
    plt.title(f"CFG Scale: {weight}")
    plt.show()

plot_cfg(1.0)
plot_cfg(4.0)

### Observation
At **CFG 1.0**, the result is exactly the Conditioned vector.
At **CFG 4.0**, the result shoots way past the original target. This is why high CFG makes images look "intense" or "oversaturated".

## 2. Real Pipeline Test

Let's see the visual effect on an actual image.

In [ ]:
from core.pipeline import pipeline_manager
pipe = pipeline_manager.get_txt2img_pipeline()

prompt = "a magical glowing mushroom"
scales = [1.0, 7.0, 20.0]

import torch
gen = torch.Generator(device).manual_seed(42)

for s in scales:
    img = pipe(prompt, guidance_scale=s, num_inference_steps=25, generator=gen).images[0]
    notebook_utils.show_image(img, title=f"CFG Scale {s}")